In [4]:
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import numpy as np
import torch

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

import timm
model = timm.create_model('efficientnet_b0', pretrained=True)
model.eval()

/home/tim/miniconda3/envs/edsr/lib/python3.6/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


{'input_size': (3, 224, 224),
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'crop_pct': 0.875}

In [104]:
# tiny-imagenet was only annotated with wnids like n012345678
# but timm's classifiers give simple ids 

# this file maps the two
wnids_to_class_idx = dict()
with open("wnids_to_class_idx.txt", 'r') as wnids_to_class_idx_file:
    contents = wnids_to_class_idx_file.readlines()
    for line in contents:
        line = line.strip().split(' ')
        wnids_to_class_idx[line[0]] = line[1]

In [123]:
# maps the tiny-imagenet wnids to the simple ids from above
annotations = dict()
with open("val_annotations.txt", 'r') as val_annotations:
    contents = val_annotations.readlines()
    i = 0
    for line in contents:
        line = line.split('\t')
        annotations[i] = wnids_to_class_idx[line[1]]
        i += 1

In [153]:
# run_name: csv filename prefix
# base_path: folder where images are located
# images should be named like this: f"{fname_start}{i}{fname_end}"
# that is, fname_start, followed by the index (no padding, i.e. 0, 1, 2,..., 10, ...), followed by fname_end with no spaces

# writes a CSV file f"{run_name}-results.csv" where each row is
# filename, ground truth label, top 5 labels predicted, top 5 probabilities, was top 1 correct?, was one of top 5 correct?
def do_classifier_run(run_name, base_path, fname_start, fname_end):
    with open(f"{run_name}-results.csv", 'w') as outfile:
        for i in range(10000):
            fname = f"{fname_start}{i}{fname_end}"
            tensor = transform(Image.open(f"{base_path}{fname}")).unsqueeze(0)    
            with torch.no_grad():
                out = model(tensor)
            probabilities = torch.nn.functional.softmax(out[0], dim=0)
            top5_prob, top5_catid = torch.topk(probabilities, 5)

            top5_prob = ["{0:0.5f}".format(i) for i in top5_prob.tolist()]
            top5_prob_string = ','.join(top5_prob)

            top5_catid = [str(j) for j in top5_catid.tolist()]
            top5_catid_string = ','.join(top5_catid)

            top1_correct = annotations[i] == top5_catid[0]
            top5_correct = annotations[i] in top5_catid

            print(f"{fname},{annotations[i]},{top5_catid_string},{top5_prob_string},{top1_correct},{top5_correct}", file=outfile)
            if i % 1000 == 0:
                print(f"done {i}/10000")

In [156]:
#do_classifier_run(run_name = "tiny-imagenet", base_path = "EDSR-PyTorch/test/", fname_start ="val_", fname_end=".png")
do_classifier_run(run_name = "pretrained_model", base_path = "EDSR-PyTorch/experiment/test/results-Demo/", fname_start ="val_", fname_end="_x4_SR.png")


done 0/10000
done 1000/10000
done 2000/10000
done 3000/10000
done 4000/10000
done 5000/10000
done 6000/10000
done 7000/10000
done 8000/10000
done 9000/10000
